<a href="https://colab.research.google.com/github/mertege/Thesis_Experiments/blob/main/Spectrogram_CI4R_Datasets_Multi_Class_CNN_24Ghz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Image-based architecure is running with only 24GHz in Gürbüz dataset.
# 5-blocks are used. Reslotion 128x128
# It was run 3-times. Results are shared on below:
# Min test accuracy is 0.733, Max test accuracy is 0.753, Mean test accuracy is 0.741, std of test accuracy is 0.009.
# Time elapsed during training time: 92.041 sec
# Time elapsed during test time: 0.309 sec

In [2]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate
from tensorflow.keras import regularizers
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K 
import gc

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 24 GHz
bending_24GHz                 = np.load('/content/drive/MyDrive/bending_24GHz.npy')
label_bending_24GHz           = np.zeros((bending_24GHz.shape[0],1))
label_bending_24GHz[:]        = 1
crawling_24GHz                = np.load('/content/drive/MyDrive/crawling_24GHz.npy')
label_crawling_24GHz          = np.zeros((crawling_24GHz.shape[0],1))
label_crawling_24GHz[:]       = 2
kneeling_24GHz                = np.load('/content/drive/MyDrive/kneeling_24GHz.npy')
label_kneeling_24GHz          = np.zeros((kneeling_24GHz.shape[0],1))
label_kneeling_24GHz[:]       = 3
limping_with_RL_Stiff_24GHz   = np.load('/content/drive/MyDrive/limping with right stiff_24GHz.npy')
label_limping_with_RL_Stiff_24GHz           = np.zeros((limping_with_RL_Stiff_24GHz.shape[0],1))
label_limping_with_RL_Stiff_24GHz[:]        = 4
picking_up_an_object_24GHz    = np.load('/content/drive/MyDrive/picking up object_24GHz.npy')
label_picking_up_an_object_24GHz          = np.zeros((picking_up_an_object_24GHz.shape[0],1))
label_picking_up_an_object_24GHz[:]       = 5
scissors_gait_24GHz           = np.load('/content/drive/MyDrive/scissors gait_24GHz.npy')
label_scissors_gait_24GHz          = np.zeros((scissors_gait_24GHz.shape[0],1))
label_scissors_gait_24GHz[:]       = 6
short_steps_24GHz             = np.load('/content/drive/MyDrive/short steps_24GHz.npy')
label_short_steps_24GHz          = np.zeros((short_steps_24GHz.shape[0],1))
label_short_steps_24GHz[:]       = 7
sitting_24GHz                 = np.load('/content/drive/MyDrive/sitting_24GHz.npy')
label_sitting_24GHz          = np.zeros((sitting_24GHz.shape[0],1))
label_sitting_24GHz[:]       = 8
walking_away_from_Radar_24GHz = np.load('/content/drive/MyDrive/walking away from radar_24GHz.npy')
label_walking_away_from_Radar_24GHz         = np.zeros((walking_away_from_Radar_24GHz.shape[0],1))
label_walking_away_from_Radar_24GHz[:]       = 9
Walking_on_both_toes_24GHz    = np.load('/content/drive/MyDrive/walking on toes both_24GHz.npy')
label_Walking_on_both_toes_24GHz         = np.zeros((Walking_on_both_toes_24GHz.shape[0],1))
label_Walking_on_both_toes_24GHz[:]       = 10
Walking_towards_radar_24GHz   = np.load('/content/drive/MyDrive/walking towards radar_24GHz.npy')
label_Walking_towards_radar_24GHz          = np.zeros((Walking_towards_radar_24GHz.shape[0],1))
label_Walking_towards_radar_24GHz[:]       = 11

In [7]:
spectrogram_concat = np.concatenate((bending_24GHz,crawling_24GHz),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,kneeling_24GHz               ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,limping_with_RL_Stiff_24GHz  ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,picking_up_an_object_24GHz   ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,scissors_gait_24GHz          ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,short_steps_24GHz            ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,sitting_24GHz                ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,walking_away_from_Radar_24GHz),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,Walking_on_both_toes_24GHz   ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,Walking_towards_radar_24GHz  ),axis=0)

label_concat = np.concatenate((label_bending_24GHz,label_crawling_24GHz),axis=0)
label_concat = np.concatenate((label_concat,label_kneeling_24GHz               ),axis=0)
label_concat = np.concatenate((label_concat,label_limping_with_RL_Stiff_24GHz  ),axis=0)
label_concat = np.concatenate((label_concat,label_picking_up_an_object_24GHz   ),axis=0)
label_concat = np.concatenate((label_concat,label_scissors_gait_24GHz          ),axis=0)
label_concat = np.concatenate((label_concat,label_short_steps_24GHz            ),axis=0)
label_concat = np.concatenate((label_concat,label_sitting_24GHz                ),axis=0)
label_concat = np.concatenate((label_concat,label_walking_away_from_Radar_24GHz),axis=0)
label_concat = np.concatenate((label_concat,label_Walking_on_both_toes_24GHz   ),axis=0)
label_concat = np.concatenate((label_concat,label_Walking_towards_radar_24GHz  ),axis=0)

In [8]:
del bending_24GHz
del crawling_24GHz
del kneeling_24GHz
del limping_with_RL_Stiff_24GHz
del picking_up_an_object_24GHz
del scissors_gait_24GHz
del short_steps_24GHz
del sitting_24GHz
del walking_away_from_Radar_24GHz
del Walking_on_both_toes_24GHz
del Walking_towards_radar_24GHz


In [9]:
# One-Hot Encode Label
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label_concat)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded)
spectrogram_concat_label = onehot_encoded_labels

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, spectrogram_concat.shape[0]), spectrogram_concat.shape[0]) # split validation data
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]
del spectrogram_concat
del spectrogram_concat_label

In [11]:
range_doppler_concat_shuffle = spectrogram_concat_shuffle
del spectrogram_concat_shuffle
range_doppler_concat_label = label_concat
del label_concat
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]

In [12]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
def mixup_augmentation(range_doppler_training_data, labels, repeat_of_mixup, alpha=0.2):
    batch_size = range_doppler_training_data.shape[0]
    concat_images = np.zeros((batch_size*(repeat_of_mixup+1),range_doppler_training_data.shape[1],range_doppler_training_data.shape[2],range_doppler_training_data.shape[3]))
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),range_doppler_training_data.shape[1],range_doppler_training_data\
                                            .shape[2],range_doppler_training_data.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))
    
    for ii in range(repeat_of_mixup):
      # shuffle train dataset
      shuffle_indx_1 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
      images_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
      range_doppler_training_data_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
      labels_shuffled_1 = labels[shuffle_indx_1,:]

      shuffle_indx_2 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
      images_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
      range_doppler_training_data_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
      labels_shuffled_2 = labels[shuffle_indx_2,:]

      # Sample lambda and reshape it to do the mixup
      ll = np.random.beta(alpha, alpha, (batch_size,1,1,1))
      x_l = np.reshape(ll, (batch_size,1,1,1))
      y_l = np.reshape(ll, (batch_size,1))
      
      # Perform mixup on both images and labels by combining a pair of images/labels
      images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * (1 - x_l)
      labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * (1 - y_l)
      concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
      concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

    concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = range_doppler_training_data
    concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images_range_doppler, concat_label)

def split_and_augmentation_of_training(range_doppler_concat_shuffle_train,range_doppler_concat_label_shuffle_train,\
                                       repeat_of_mixup, augmentation_enable):
  # ---------------- Parameters ----------------
  size_of_validation = 50
  # Split validation
  randomlist_for_validation_indx = random.sample(range(0, range_doppler_concat_shuffle_train.shape[0]), size_of_validation) # split validation data
  randomlist_for_train_indx = np.delete(range(0, range_doppler_concat_shuffle_train.shape[0]), randomlist_for_validation_indx) # split training data
  # get validation data
  spectrogram_validation_labels = range_doppler_concat_label_shuffle_train[randomlist_for_validation_indx,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  # get training data
  spectrogram_training_labels = spectrogram_concat_label_shuffle_train[randomlist_for_train_indx,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]

  (range_doppler_augmented_image,spectrograms_label)=\
   mixup_augmentation(range_doppler_training_data, spectrogram_training_labels, repeat_of_mixup, alpha=0.2)

  return (range_doppler_augmented_image,spectrograms_label,\
     range_doppler_validation_data, spectrogram_validation_labels)



def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


In [13]:
range_doppler_concat_shuffle_new = np.zeros((range_doppler_concat_shuffle.shape[0],128,128,1))
for ii in range(range_doppler_concat_shuffle.shape[0]):
  aa = range_doppler_concat_shuffle[ii,:,:,0]
  range_doppler_concat_shuffle_new[ii,:,:,0] = cv2.resize(aa, (range_doppler_concat_shuffle_new.shape[2], range_doppler_concat_shuffle_new.shape[1]),interpolation = cv2.INTER_CUBIC)
range_doppler_concat_shuffle = range_doppler_concat_shuffle_new
del aa
del range_doppler_concat_shuffle_new

In [14]:
normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)


In [15]:

# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 128
dropout_prob_cnn = 0.1
dropout_prob_dense = 0.3
repeat_of_mixup = 2
number_of_repeat = 3
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  if repeat_run_number > 0:
    del range_doppler_concat_shuffle_test
    del range_doppler_augmented_image
    del range_doppler_concat_shuffle_train
   
  for randomlist_for_train_indx, randomlist_for_test_indx in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):   
    gc.collect()
    K.clear_session()
    
    # test data
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    range_doppler_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_test_indx,:]
    #train data
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
      # ---------------- MixUp Augmentation ----------------
    (range_doppler_augmented_image,spectrogram_concat_label_shuffle_concat,\
     validation_range_doppler, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(range_doppler_concat_shuffle_train,\
                                         spectrogram_concat_label_shuffle_train,\
                                         repeat_of_mixup, augmentation_enable)
    
    # ---------------- Neural Network Architecture ----------------

    def encoder_for_range_doppler(input_shape):
      input = Input(shape=input_shape)
      x = Conv2D(4, (3, 3),padding='same',kernel_initializer="glorot_normal")(input)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(8, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(0.2)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)   
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(0.2)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)     
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(0.2)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)           
      x = Flatten()(x)
      return Model(input, x)


    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(128,kernel_regularizer='l2')(input)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(0.5)(x)
      x = Dense(dense_size,kernel_regularizer='l2')(x)
      # x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(11, activation="softmax")(x)
      return Model(input, x)
    input_shape = range_doppler_concat_shuffle.shape[1:]
    base_network_range_doppler = encoder_for_range_doppler(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_range_doppler(range_doppler_input)


    base_decoder_network = decoder_for_concat((processed_range_doppler.shape[1]))
    out = base_decoder_network(processed_range_doppler)

    model = Model(inputs=[range_doppler_input], outputs=[out]) 
    print(model.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=35, verbose=0,restore_best_weights=True, mode='min')
    t = time.time()
    history = model.fit((range_doppler_augmented_image),(spectrogram_concat_label_shuffle_concat),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    verbose=0,
                    callbacks=[earlyStopping],
                    validation_data = ((validation_range_doppler) , (spectrogram_validation_labels)))
    elapsed = time.time() - t
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test],\
                                               [range_doppler_concat_label_shuffle_test],
                  batch_size=batch_size)

    test_accuracy_per_fold.append(test_accuracy)
    # f1_score_per_fold.append(test_f1_score)

  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  # f1_score_per_run.append(sum(f1_score_per_fold)/num_folds)
  print(test_accuracy_per_run)
  # print(f1_score_per_run)
print(f'Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
max test accuracy is {"{:.2f}".format(max(test_accuracy_per_run))},\
min test accuracy is {"{:.2f}".format(min(test_accuracy_per_run))}, \
std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_run, axis=0))}.')
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 model (Functional)          (None, 256)               6384      
                                                                 
 model_1 (Functional)        (None, 11)                51339     
                                                                 
Total params: 57,723
Trainable params: 57,347
Non-trainable params: 376
_________________________________________________________________
None
12/12 [==============================] - 0s 7ms/step - loss: 0.9761 - accuracy: 0.7546
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 1)]     0         
          

In [16]:
t_test = time.time()
y_test_predicted = model.predict((range_doppler_concat_shuffle_test[0:1,:,:,:]))
test_elapsed = time.time() - t_test 
print(f'Time elapsed during test time: {"{:.2f}".format(test_elapsed)}, sec')

Time elapsed during test time: 0.31, sec


In [17]:
print(f'Min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
Max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))}, \
Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, \
std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
print(f'Time elapsed during training time: {"{:.3f}".format(elapsed)} sec')
print(f'Time elapsed during test time: {"{:.3f}".format(test_elapsed)} sec')

Min test accuracy is 0.733, Max test accuracy is 0.753, Mean test accuracy is 0.741, std of test accuracy is 0.009.
Time elapsed during training time: 92.041 sec
Time elapsed during test time: 0.309 sec
